In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import nltk

# Assicurati di avere i token di NLTK per il tokenization
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ANGELAANGUILANO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:

# Dataset di esempio
corpus = [
    "Questo è un esempio di generazione di testo.",
    "Le reti neurali sono molto potenti per il processing del linguaggio naturale.",
    "Le RNN possono essere utilizzate per la previsione della parola successiva.",
    "Questo è un esempio semplice di come usare una RNN per generare parole."
]

# Preprocessamento del testo: Tokenizzazione
def preprocess_text(corpus):
    # Tokenizza il testo e unisci tutto in un'unica lista di parole
    text = ' '.join(corpus)
    tokens = word_tokenize(text.lower())  # Tokenizzazione
    return tokens

tokens = preprocess_text(corpus)

# Creazione di un vocabolario tramite Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([tokens])
total_words = len(tokenizer.word_index) + 1  # Aggiungiamo 1 per l'indexing zero

# Creazione delle sequenze di input e target
input_sequences = []
for i in range(1, len(tokens)):
    n_gram_sequence = tokens[i-1:i+1]  # Crea una sequenza di due parole
    input_sequences.append(n_gram_sequence)

# Creazione degli input e degli output per l'addestramento
X = []
y = []

for seq in input_sequences:
    prev_word = tokenizer.texts_to_sequences([seq[0]])  # Parola precedente
    next_word = tokenizer.texts_to_sequences([seq[1]])  # Parola successiva
    
    # Verifica che entrambe le parole siano presenti nel vocabolario
    if prev_word and next_word and len(prev_word[0]) > 0 and len(next_word[0]) > 0:
        X.append(prev_word[0][0])  # Converte la parola precedente in indice numerico
        y.append(next_word[0][0])  # Converte la parola successiva in indice numerico

# Conversione degli input e degli output in array NumPy
X = np.array(X)
y = np.array(y)

# Reshape di X per aggiungere la dimensione "timesteps"
X = X.reshape((X.shape[0], 1))  # Ogni sequenza ha un solo passo temporale (parola)

# Convertiamo y in una rappresentazione one-hot encoded
y = to_categorical(y, num_classes=total_words)

In [3]:

# Costruzione del modello RNN classico
model = Sequential()
model.add(Embedding(total_words, 50, input_length=1))  # L'input è una singola parola
model.add(SimpleRNN(100, return_sequences=False))  # RNN classica
model.add(Dense(total_words, activation='softmax'))

# Compilazione del modello
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Addestramento del modello
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100


C:\Users\ANGELAANGUILANO\simple-environment\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.0750 - loss: 3.5826
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1250 - loss: 3.5646
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.1500 - loss: 3.5496 
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3250 - loss: 3.5349
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.3500 - loss: 3.5205 
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 3.5066 
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5500 - loss: 3.4920 
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.6000 - loss: 3.4771 
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6500 - loss: 3.4622
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6500 - loss: 3.4468
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7000 - loss: 3.4309 
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7000 - loss: 3.

In [6]:
import numpy as np

def predict_next_word(model, tokenizer, text):
    # Tokenizza il testo di input e convertilo in sequenza numerica
    tokenized_input = tokenizer.texts_to_sequences([text.lower().split()])
    
    # Reshape per il modello RNN (aggiungiamo la dimensione "timesteps")
    tokenized_input = np.array(tokenized_input)
    tokenized_input = tokenized_input.reshape((1, tokenized_input.shape[1]))  # Reshaping to (1, sequence_length)
    
    # Predizione della parola successiva
    predicted_probs = model.predict(tokenized_input)
    
    # Prendi la parola con la probabilità massima
    predicted_word_index = np.argmax(predicted_probs)
    
    # Converti l'indice della parola in parola
    predicted_word = tokenizer.index_word[predicted_word_index]
    return predicted_word


In [7]:
input_text = "le reti"
predicted_word = predict_next_word(model, tokenizer, input_text)
print(f"Parola successiva prevista per '{input_text}': {predicted_word}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
Parola successiva prevista per 'le reti': neurali
